<a href="https://colab.research.google.com/github/Marcin19721205/BDCaseStudy/blob/main/BDCSZad5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Zadanie 5

1. prosty model predykcyjny, który ma oszacować prawdopodobieństwo zakupu (Purchase=1) dla każdego klienta. Na wejściu wszystkie sensowne cechy z pliku Tayko, ale świadomie wyrzycam trzy kolumny: Purchase - używany jako  target, sequence_number - to tylko identyfikator/kolejnośćoraz Spending. (kluczowy, bo jest kopią targetu — w danych niemal zawsze Spending > 0 oznacza Purchase = 1, więc użycie go byłoby zwykłym „dtata leak” (model wyglada hiper, ale tylko dlatego, że ma target w train

2. Dane podzielone na zbiór treningowy i testowy  60/40 (losowo, z ziarnem, żeby wynik był powtarzalny). prewencyjnie braki danych uzupełniam medianą, a następnie standaryzuję cechy TRAIN, żeby regresja logistyczna pracowała na porównywalnych skalach. Na tej podstawie uczę jeden model regresji logistycznej i na zbiorze testowym wyznaczam proba = P(Purchase=1), czyli wynik od 0 do 1, który traktuję jako ranking klientów. Skuteczność rankingu pokazuję później na wykresach Lift i Cumulative Gains, bo w tym zastosowaniu ważniejsze jest „kogo brać w pierwszej kolejności”, niż sama klasyfikacja przy progu 0.5.

In [21]:
import pandas as pd
import numpy as np

In [22]:
path = "sample_data/Tayko.csv"  # plik z zadania
df = pd.read_csv(path)  # load
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 25 columns):
 #   Column                Non-Null Count  Dtype
---  ------                --------------  -----
 0   sequence_number       2000 non-null   int64
 1   US                    2000 non-null   int64
 2   source_a              2000 non-null   int64
 3   source_c              2000 non-null   int64
 4   source_b              2000 non-null   int64
 5   source_d              2000 non-null   int64
 6   source_e              2000 non-null   int64
 7   source_m              2000 non-null   int64
 8   source_o              2000 non-null   int64
 9   source_h              2000 non-null   int64
 10  source_r              2000 non-null   int64
 11  source_s              2000 non-null   int64
 12  source_t              2000 non-null   int64
 13  source_u              2000 non-null   int64
 14  source_p              2000 non-null   int64
 15  source_x              2000 non-null   int64
 16  source

In [23]:
df.head()

,sequence_number,US,source_a,source_c,source_b,source_d,source_e,source_m,source_o,source_h,...,source_x,source_w,Freq,last_update_days_ago,1st_update_days_ago,Web order,Gender=male,Address_is_res,Purchase,Spending
0,1,1,0,0,1,0,0,0,0,0,...,0,0,2,3662,3662,1,0,1,1,128
1,2,1,0,0,0,0,1,0,0,0,...,0,0,0,2900,2900,1,1,0,0,0
2,3,1,0,0,0,0,0,0,0,0,...,0,0,2,3883,3914,0,0,0,1,127
3,4,1,0,1,0,0,0,0,0,0,...,0,0,1,829,829,0,1,0,0,0
4,5,1,0,1,0,0,0,0,0,0,...,0,0,1,869,869,0,0,0,0,0


In [24]:
# sanity-check + rozkład targetu Purchase
shape = df.shape  # (N, p)
dtypes = df.dtypes.value_counts().to_dict()  # typy kolumn
na_total = int(df.isna().sum().sum())  # braki ogółem
na_by_col = df.isna().sum().sort_values(ascending=False)  # braki per kolumna
dup_rows = int(df.duplicated().sum())  # duplikaty wierszy

target = "Purchase"  # target
purchase_dtype = df[target].dtype  # typ targetu
purchase_counts = df[target].value_counts(dropna=False).sort_index()  # liczności
purchase_pct = (df[target].value_counts(normalize=True, dropna=False).sort_index() * 100).round(2)  # %
baseline_rr = float(df[target].mean())  # odsetek klasy 1 (baseline)

print("\nSANITY-CHECK  ===")
print(f"Shape: {shape} | Duplikaty wierszy: {dup_rows} | Braki ogółem: {na_total}")
print(f"Typy kolumn: {dtypes}")
print("\nBraki per kolumna (top 10):")
print(na_by_col.head(10).to_string())

print("\nTARGET: Purchase ===")
print(f"Typ: {purchase_dtype} | Unikalne: {sorted(df[target].dropna().unique().tolist())}")
print(pd.DataFrame({"count": purchase_counts, "pct": purchase_pct}).to_string())
print(f"Baseline (mean(Purchase)=P(Purchase=1)): {baseline_rr:.4f}")



SANITY-CHECK  ===
Shape: (2000, 25) | Duplikaty wierszy: 0 | Braki ogółem: 0
Typy kolumn: {dtype('int64'): 25}

Braki per kolumna (top 10):
sequence_number    0
US                 0
source_a           0
source_c           0
source_b           0
source_d           0
source_e           0
source_m           0
source_o           0
source_h           0

TARGET: Purchase ===
Typ: int64 | Unikalne: [0, 1]
          count   pct
Purchase             
0          1000  50.0
1          1000  50.0
Baseline (mean(Purchase)=P(Purchase=1)): 0.5000


In [25]:
# walidacja leakage: Spending>0 <==> Purchase=1 + decyzja (Spending OUT
has_spending = "Spending" in df.columns  # czy jest Spending
has_purchase = "Purchase" in df.columns  # czy jest Purchase

if has_spending and has_purchase:
    chk = df.loc[df["Spending"].notna() & df["Purchase"].notna(), ["Spending", "Purchase"]].copy()  # kompletne
    chk["sp_gt0"] = (chk["Spending"] > 0).astype(int)  # Spending>0 -> 1/0
    chk["purchase01"] = chk["Purchase"].astype(int)  # Purchase -> int

    ct = pd.crosstab(chk["sp_gt0"], chk["purchase01"], dropna=False)  # tabela zgodności
    mismatch = int((chk["sp_gt0"] != chk["purchase01"]).sum())  # ile nie pasuje
    mismatch_rate = (mismatch / len(chk)) if len(chk) else np.nan  # % niezgodności

    print("\nLEAKAGE CHECK: Spending vs Purchase ===")
    print(f"Czy są kolumny? Spending={has_spending} | Purchase={has_purchase}")
    print(f"Liczba porównań (kompletne wiersze): {len(chk)}")
    print("\nTabela zgodności (wiersze: Spending>0, kolumny: Purchase):")
    print(ct.to_string())
    print(f"\nNiezgodności: {mismatch} |  błedy %: {mismatch_rate:.4%}")

    if mismatch_rate <= 0.01:  # 1% jako twardy próg na 'praktycznie reguła'
        print("\nDecyzja: Spending USUWAMY z predyktorów (silny leakage).")
    else:
        print("\nDecyzja: Spending nie jest prostą kopią targetu (brak oczywistego leakage).")

else:
    print("\n=== LEAKAGE CHECK: pominięty ===")
    print(f"Brak wymaganych kolumn: Spending={has_spending} | Purchase={has_purchase}")



LEAKAGE CHECK: Spending vs Purchase ===
Czy są kolumny? Spending=True | Purchase=True
Liczba porównań (kompletne wiersze): 2000

Tabela zgodności (wiersze: Spending>0, kolumny: Purchase):
purchase01    0     1
sp_gt0               
0           999     0
1             1  1000

Niezgodności: 1 |  błedy %: 0.0500%

Decyzja: Spending USUWAMY z predyktorów (silny leakage).


In [26]:
# Przygotowanie cech X pod regresję logistyczną
from sklearn.compose import ColumnTransformer  # num/cat
from sklearn.pipeline import Pipeline          # spójny preprocessing
from sklearn.impute import SimpleImputer       # imputacja
from sklearn.preprocessing import OneHotEncoder # one-hot (gdyby były kategorie)

target = "Purchase"  # target 0/1

drop_cols = [
    target,             # target
    "Spending",         # leakage (pkt 2)
    "sequence_number",  # ID/sekwencja
]

drop_cols = [c for c in drop_cols if c in df.columns]  # tylko te, które istnieją

X_full = df.drop(columns=drop_cols)        # cechy
y_full = df[target].astype(int)            # target

num_cols = X_full.select_dtypes(include=["number"]).columns.tolist()              # num
cat_cols = X_full.select_dtypes(exclude=["number"]).columns.tolist()              # cat

preprocess = ColumnTransformer(
    transformers=[
        ("num", Pipeline([("imputer", SimpleImputer(strategy="median"))]), num_cols),
        ("cat", Pipeline([("imputer", SimpleImputer(strategy="most_frequent")),
                          ("onehot", OneHotEncoder(handle_unknown="ignore"))]), cat_cols),
    ],
    remainder="drop"
)


print(f"Usunite kolumny: {drop_cols}")
print(f"X: {X_full.shape} | y: {y_full.shape} | num={len(num_cols)} | cat={len(cat_cols)}")


Usunite kolumny: ['Purchase', 'Spending', 'sequence_number']
X: (2000, 22) | y: (2000,) | num=22 | cat=0


In [27]:
# Split train/test 60/40 (shuffle + random_state=123)
from sklearn.model_selection import train_test_split  # split

X_train, X_test, y_train, y_test = train_test_split(
    X_full, y_full,
    test_size=0.4,
    random_state=123,
    shuffle=True
)

print("split 60/40")
print(f"Train: X={X_train.shape}, y={y_train.shape} | Test: X={X_test.shape}, y={y_test.shape}")


split 60/40
Train: X=(1200, 22), y=(1200,) | Test: X=(800, 22), y=(800,)


In [28]:
# Standaryzacja fit na train, transform na test
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler

imp = SimpleImputer(strategy="median")     # braki -> mediana (num)
scaler = StandardScaler()                  # standaryzacja

X_train_imp = imp.fit_transform(X_train)   # fit na train
X_test_imp  = imp.transform(X_test)        # transform na test

X_train_std = scaler.fit_transform(X_train_imp)  # fit na train
X_test_std  = scaler.transform(X_test_imp)       # transform na test

print("Train shape:", X_train_std.shape, "| Test shape:", X_test_std.shape)


Train shape: (1200, 22) | Test shape: (800, 22)


In [31]:
# Logistic Regression  — trening + proba na test
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(max_iter=2000, solver="lbfgs")  #
lr.fit(X_train_std, y_train)  # trening

proba_test = lr.predict_proba(X_test_std)[:, 1]  # P(Purchase=1) na test

print("proba_test head:", proba_test[:10])


proba_test head: [0.92519275 0.93969986 0.80489032 0.02584829 0.0502189  0.56357162
 0.64927031 0.22387603 0.73323368 0.57994123]


In [32]:
# Lift chart (TEST) — LR proba_test; decyle; lift=rr_decyl/rr_overall_test; etykiety + baseline
import numpy as np
import pandas as pd
import plotly.express as px

rr_overall_test = float(y_test.mean())  # baseline na teście

t = pd.DataFrame({"Purchase": y_test.values, "proba": proba_test})  # y + score
t = t.sort_values("proba", ascending=False).reset_index(drop=True)  # sort malejąco

t["decile"] = (np.floor(np.arange(len(t)) / (len(t) / 10)) + 1).astype(int)  # 1..10
t["decile"] = t["decile"].clip(1, 10)  # safety

lift_tbl = (t.groupby("decile")
            .agg(customers=("Purchase", "size"),
                 purchases=("Purchase", "sum"))
            .reset_index())

lift_tbl["rr_decile"] = lift_tbl["purchases"] / lift_tbl["customers"]  # rr w decylu
lift_tbl["rr_overall_test"] = rr_overall_test  # baseline
lift_tbl["lift"] = lift_tbl["rr_decile"] / lift_tbl["rr_overall_test"]  # lift

lift_tbl = lift_tbl.sort_values("decile").reset_index(drop=True)  # 1..10
lift_tbl["txt"] = (
    "n=" + lift_tbl["customers"].astype(int).astype(str)
    + "<br>buy=" + lift_tbl["purchases"].astype(int).astype(str)
    + "<br>rr=" + lift_tbl["rr_decile"].round(4).astype(str)
)

fig = px.bar(
    lift_tbl,
    x="decile",
    y="lift",
    text="txt",
    hover_data=["customers", "purchases", "rr_decile", "rr_overall_test"],
    title=f"Lift Chart (TEST) — Logistic Regression | rr_overall_test={rr_overall_test:.4f} | baseline lift=1.0",
    template="plotly_dark"
)
fig.update_traces(textposition="outside", cliponaxis=False)
fig.add_hline(y=1.0, annotation_text="baseline (lift=1.0)", annotation_position="top left")
fig.update_layout(
    xaxis_title="Decyl (1 = najwyższe P(Purchase=1), 10 = najniższe)",
    yaxis_title="Lift = rr_decyl / rr_overall_test",
    uniformtext_minsize=8,
    uniformtext_mode="hide"
)
fig.show()

lift_tbl


,decile,customers,purchases,rr_decile,rr_overall_test,lift,txt
0,1,80,79,0.9875,0.51625,1.912833,n=80<br>buy=79<br>rr=0.9875
1,2,80,73,0.9125,0.51625,1.767554,n=80<br>buy=73<br>rr=0.9125
2,3,80,71,0.8875,0.51625,1.719128,n=80<br>buy=71<br>rr=0.8875
3,4,80,57,0.7125,0.51625,1.380145,n=80<br>buy=57<br>rr=0.7125
4,5,80,52,0.6500,0.51625,1.259080,n=80<br>buy=52<br>rr=0.65
5,6,80,34,0.4250,0.51625,0.823245,n=80<br>buy=34<br>rr=0.425
6,7,80,28,0.3500,0.51625,0.677966,n=80<br>buy=28<br>rr=0.35
7,8,80,15,0.1875,0.51625,0.363196,n=80<br>buy=15<br>rr=0.1875
8,9,80,4,0.0500,0.51625,0.096852,n=80<br>buy=4<br>rr=0.05
9,10,80,0,0.0000,0.51625,0.000000,n=80<br>buy=0<br>rr=0.0


In [33]:
# Cumulative gains chart (TEST) — LR proba_test; sort po proba; baseline losowy
import numpy as np
import pandas as pd
import plotly.express as px

t = pd.DataFrame({"Purchase": y_test.values, "proba": proba_test})  # y + score
t = t.sort_values("proba", ascending=False).reset_index(drop=True)  # sort malejąco

t["cum_pop"] = (np.arange(len(t)) + 1) / len(t)  # % populacji (0..1)
total_pos = int(t["Purchase"].sum())  # ile zakupów w teście
t["cum_tp"] = t["Purchase"].cumsum()  # kumulacja zakupów
t["cum_gain"] = (t["cum_tp"] / total_pos) if total_pos > 0 else 0.0  # % przechwyconych zakupów

gains = t[["cum_pop", "cum_gain"]].copy()  # krzywa modelu
gains["model"] = "LogReg (LR)"  # etykieta

rand = pd.DataFrame({  # linia losowa
    "cum_pop": np.linspace(0, 1, 101),
    "cum_gain": np.linspace(0, 1, 101),
    "model": "Losowo (baseline)"
})

g_long = pd.concat([gains, rand], ignore_index=True)  # long

fig = px.line(
    g_long,
    x="cum_pop",
    y="cum_gain",
    color="model",
    title="Cumulative Gains Chart (TEST) — Logistic Regression vs baseline losowy",
    template="plotly_dark"
)
fig.update_layout(
    xaxis_title="Skumulowany % populacji (od najwyższej proba do najniższej)",
    yaxis_title="Skumulowany % zakupów (TP) przechwyconych"
)
fig.show()
